In [18]:
import random
from datetime import timedelta

import numpy as np
import pandas as pd

pd.set_option("display.max_rows", 1000)


In [22]:
def generate_dummy_data() -> pd.DataFrame:
    """
    ダミーデータを生成する関数。

    Returns:
        pd.DataFrame: ダミーデータが格納されたDataFrame。
    """
    date_index = pd.date_range(start="2023-01-01", end="2023-01-05", freq="s")

    columns = ["data1", "data2", "data3"]
    data = {}
    for col in columns:
        data.update({col: np.random.randn(len(date_index))})
    df = pd.DataFrame(index=date_index, data=data)
    df["time_delta"] = [timedelta(seconds=random.randrange(1, 10)) for _ in range(len(df))]
    df["time_delta_cumsum"] = df["time_delta"].cumsum()
    df["DateTime"] = df.index + df["time_delta_cumsum"]
    df.drop(["time_delta", "time_delta_cumsum"], axis=1, inplace=True)
    # df["time_delta"] = df["time_delta"].dt.seconds
    df.set_index("DateTime", inplace=True)

    return df


def resample_dataframe(dataframe: pd.DataFrame, interval: str = "30s") -> pd.DataFrame:
    """
    DataFrameを指定されたインターバルでリサンプリングする関数。

    Parameters:
        dataframe (pd.DataFrame): リサンプリング対象のDataFrame。
        interval (str, optional): リサンプリングするインターバル。デフォルトは"30s"。

    Returns:
        pd.DataFrame: リサンプリングされたDataFrame。
    """
    re_index = pd.date_range(start=dataframe.index.min(), end=dataframe.index.max(), freq=interval)
    re_df = pd.DataFrame(index=re_index)
    result = pd.merge_asof(
        re_df,
        dataframe,
        left_index=True,
        right_index=True,
        direction="nearest",
        tolerance=pd.Timedelta(seconds=15),
    )
    return result

In [23]:
df = create_dummy_data()
df.head(10)

,data1,data2,data3
Date_Time,,,
2023-01-01 00:00:01,-0.013299,1.540332,-1.435462
2023-01-01 00:00:04,0.956679,-0.254348,-1.503594
2023-01-01 00:00:14,-0.901492,-1.117122,-0.309230
2023-01-01 00:00:19,0.322190,1.787706,-0.843701
2023-01-01 00:00:26,1.449751,-0.445148,-0.635967
2023-01-01 00:00:35,0.622199,1.367670,0.557008
2023-01-01 00:00:42,0.037041,-0.740938,1.282642
2023-01-01 00:00:52,-1.556792,-1.661210,0.125919
2023-01-01 00:00:55,1.100434,-1.390560,0.160198


In [24]:
resample_df = resample_dataframe(df)
resample_df.head(20)

,data1,data2,data3
2023-01-01 00:00:01,-0.013299,1.540332,-1.435462
2023-01-01 00:00:31,0.622199,1.367670,0.557008
2023-01-01 00:01:01,-0.443583,1.144472,0.539733
2023-01-01 00:01:31,1.006398,0.178246,1.041656
2023-01-01 00:02:01,1.439442,0.105630,-0.483438
2023-01-01 00:02:31,-0.698991,-1.618728,-1.511227
2023-01-01 00:03:01,1.387302,-1.383429,0.578768
2023-01-01 00:03:31,0.395004,-0.790624,0.748879
2023-01-01 00:04:01,-0.302743,0.019650,0.139199
2023-01-01 00:04:31,0.267218,1.460378,0.512358
